In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
#print(os.listdir(os.path.join('gdrive', 'MyDrive', 'research')))
os.chdir(os.path.join('gdrive', 'MyDrive'))
print(os.listdir())

['weatherAUS.csv', 'Colab Notebooks', 'Qianwei Yu | qy824@nyu.edu.gslides', 'presentation.gslides', 'Cross culture fusion.gslides', 'Copy of Outreach Template.docx', 'EAP 101 Qianwei Yu (Amy)', 'N19816638.jpg', "PeiQuan Feng(Glen pf2208)'s summary on an article about cultural appropriation.mp4", '历保', 'Leasing team.mov', 'Qianwei Yu, Precalculus, 2022 Fall', 'Copy of Course Plan.gsheet', 'ED3A3_STATISTICAL_REPORT_G9.docx', 'Copy of wine_data.csv', 'Fall 2022 Business Analytics', 'assignment5_data.csv', 'Copy of emotion_train_text.txt', 'Copy of emotion_train_labels.txt', 'Copy of emotion_test_text.txt', 'Copy of emotion_test_labels.txt', 'Copy of NLP lab template.ipynb', 'Shanghai previous year data.csv', 'Paris historical data with FastPass.csv', 'Copy of regression.ipynb', 'Copy of vehicle_purchase_data.csv', 'Copy of output_dataset.csv', 'Copy of Copy of NLP lab template.ipynb', 'paris_data.csv', 'Shanghai_data.csv', 'Paris_data.csv', 'output_data.csv', 'Cluster final.ipynb', 'Qianw

In [ ]:
!python -m pip install "dask[complete]" >& /dev/null   # Install everything
!pip install geopandas

In [ ]:
import dask
import dask.dataframe as dd
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

In [ ]:
cities = ['shanghai', 'beijing', 'guangzhou', 'chongqing']
groups = [u'全部', u'18岁以下', u'60岁以上', u'低收入人群']

city = cities[0]
group = groups[1]

#txt_file_path = '/content/drive/MyDrive/Work With RA/CLUEs_2023_Summer/Project__UrbanHeatExposure_Mobility/Heatwave OD data set/'

date = pd.read_csv('heatwave_dates.csv')
date.head()

,city,date_dt,heatwave
0,shanghai,20190724,True
1,shanghai,20190725,True
2,shanghai,20190726,True
3,shanghai,20190727,True
4,shanghai,20190728,True


In [ ]:
#txt_file_path_OD = txt_file_path + '数据/20230728/'
#txt_file_name = txt_file_path_OD + 'shnydx_odpf_' + city + '_2019.txt'

# Convert to parquet (comment after the 1st run)
# df = dd.read_csv(txt_file_name, delimiter='\t')
# df.to_parquet(txt_file_path + 'shnydx_odpf_' + str(city) + '_2019/')

# Read parquet format
df = dd.read_parquet('shnydx_odpf_shanghai_2019')
df.head()

,flag,date_dt,start_grid,end_grid,pop,odleng
0,全部,20190724,310100166,3101007038,9.0,8.8
1,全部,20190724,310100166,31010044923,4.0,30.0
2,全部,20190724,310100166,310100110080,4.0,60.4
3,全部,20190724,3101001083,3101009291,4.0,4.5
4,全部,20190724,3101001083,310100133403,4.0,75.2


In [ ]:
#txt_file_path_grid = txt_file_path + u'图层/' + city + '_grid.gpkg'
gdf = gpd.read_file('shanghai_grid.gpkg')
gdf['gridid'] = gdf['gridid'].astype(int)
gdf['outflow_p_value'] = 1
gdf['outflow_norm'] = 0
gdf['outflow_heat'] = 0
gdf['outflow_diff'] = 0
gdf['inflow_p_value'] = 1
gdf['inflow_norm'] = 0
gdf['inflow_heat'] = 0
gdf['inflow_diff'] = 0
gdf.dtypes

,0
Id,int32
Tid,int64
Province,object
Zone,object
gridid,int64
grid_area,float64
grid_x,float64
grid_y,float64
geometry,geometry
outflow_p_value,int64


In [ ]:
outflows = df[df['flag']==group][['start_grid','date_dt','pop']]\
  .groupby(['start_grid', 'date_dt'])\
  .agg({'pop': 'sum'})\
  .compute().reset_index()

inflows = df[df['flag']==group][['end_grid','date_dt','pop']]\
  .groupby(['end_grid', 'date_dt'])\
  .agg({'pop': 'sum'})\
  .compute().reset_index()

# outflows.head()
outflows_dt = pd.merge(outflows, date, on='date_dt')
outflows_dt.head()

inflows_dt = pd.merge(inflows, date, on='date_dt')
inflows_dt.head()

,end_grid,date_dt,pop,city,heatwave
0,31010014322,20190724,70.0,shanghai,True
1,31010014322,20190724,70.0,beijing,True
2,3101005657,20190724,36.0,shanghai,True
3,3101005657,20190724,36.0,beijing,True
4,31010012508,20190724,130.0,shanghai,True


In [ ]:
from scipy.stats import f_oneway

for gridid in tqdm(gdf['gridid']):
  gridid = int(gridid)
  # Gather outflows of the grid
  outflows_dt_grid = outflows_dt[outflows_dt['start_grid']==gridid]
  outflows_dt_pop_heat = outflows_dt_grid[outflows_dt_grid['heatwave']==True]['pop']
  outflows_dt_pop_norm = outflows_dt_grid[outflows_dt_grid['heatwave']==False]['pop']
  # Gather inflows of the grid
  inflows_dt_grid = inflows_dt[inflows_dt['end_grid']==gridid]
  inflows_dt_pop_heat = inflows_dt_grid[inflows_dt_grid['heatwave']==True]['pop']
  inflows_dt_pop_norm = inflows_dt_grid[inflows_dt_grid['heatwave']==False]['pop']
  # Process outflows
  if((len(outflows_dt_pop_heat)>0) & (len(outflows_dt_pop_norm)>0)):
    F_stat, p_value = f_oneway(outflows_dt_pop_heat, outflows_dt_pop_norm)
    gdf.loc[gdf['gridid']==gridid, ['outflow_p_value']] = p_value
    gdf.loc[gdf['gridid']==gridid, ['outflow_diff']] = outflows_dt_pop_heat.mean() - outflows_dt_pop_norm.mean()
    gdf.loc[gdf['gridid']==gridid, ['frc_outflow_diff']] = (outflows_dt_pop_heat.mean() - outflows_dt_pop_norm.mean())/outflows_dt_pop_norm.mean()
   # Process inflows
  if((len(inflows_dt_pop_heat)>0) & (len(inflows_dt_pop_norm)>0)):
    F_stat, p_value = f_oneway(inflows_dt_pop_heat, inflows_dt_pop_norm)
    gdf.loc[gdf['gridid']==gridid, ['inflow_p_value']] = p_value
    gdf.loc[gdf['gridid']==gridid, ['inflow_diff']] = inflows_dt_pop_heat.mean() - inflows_dt_pop_norm.mean()
    gdf.loc[gdf['gridid']==gridid, ['frc_inflow_diff']] = (inflows_dt_pop_heat.mean() - inflows_dt_pop_norm.mean())/inflows_dt_pop_norm.mean()

gdf.to_file('Shanghai_60_grid_pvalues.gpkg')

gdf.head()

100%|██████████| 130440/130440 [15:39<00:00, 138.87it/s]


,Id,Tid,Province,Zone,gridid,grid_area,grid_x,grid_y,geometry,outflow_p_value,outflow_norm,outflow_heat,outflow_diff,inflow_p_value,inflow_norm,inflow_heat,inflow_diff,frc_outflow_diff,frc_inflow_diff
0,0,166,031,上海市,310100166,62500.0,909111.013987,3.403343e+06,"MULTIPOLYGON (((908986.014 3403218.205, 908986...",8.803442e-07,0,0,57.080357,0.000008,0,0,53.098214,0.937940,0.874559
1,0,622,031,上海市,310100622,62500.0,909111.013987,3.403593e+06,"MULTIPOLYGON (((908986.014 3403468.205, 908986...",1.000000e+00,0,0,0.000000,1.000000,0,0,0.000000,NaN,NaN
2,0,623,031,上海市,310100623,62500.0,909361.013987,3.403593e+06,"MULTIPOLYGON (((909236.014 3403468.205, 909236...",3.554053e-02,0,0,-1.733333,0.483688,0,0,0.846154,-0.288889,0.169231
3,0,624,031,上海市,310100624,62500.0,909611.013987,3.403593e+06,"MULTIPOLYGON (((909486.014 3403468.205, 909486...",1.000000e+00,0,0,0.000000,1.000000,0,0,0.000000,NaN,NaN
4,0,1077,031,上海市,3101001077,62500.0,908861.013987,3.403843e+06,"MULTIPOLYGON (((908736.014 3403718.205, 908736...",9.678145e-01,0,0,0.104167,0.531246,0,0,-1.000000,0.011161,-0.142857
